In [ ]:
# Dependencias
import os
import cv2
import pickle # Pickle ya es parte de Python en versiones mas nuevas :)
import mediapipe as mp
import matplotlib.pyplot as plt

In [1]:
import mediapipe as mp
import cv2
import numpy as np

def calibration():
    # Parámetros
    max_calibration_samples = 100  # Número máximo de muestras para calibración
    specific_finger_indices = [4, 8, 12, 16, 20]  # Índices de las puntas de los dedos a promediar

    mp_drawing = mp.solutions.drawing_utils
    mp_hands = mp.solutions.hands

    calibration_samples = {index: [] for index in specific_finger_indices}  # Diccionario para almacenar las muestras de calibración por índice de dedo

    # Inicializar captura de video
    cap = cv2.VideoCapture(0)

    # Instanciar modelo de manos
    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
        calibration_started = False
        samples_collected = 0

        while True:
            ret, frame = cap.read()

            #DETECCION CON MEDIAPIPE
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = cv2.flip(image, 1)
            image.flags.writeable = False
            results = hands.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                              mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                              mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))

                    for index in specific_finger_indices:
                        landmark = hand_landmarks.landmark[index]
                        # Agregar coordenadas x e y de la punta del dedo al diccionario de muestras
                        calibration_samples[index].append([landmark.x, landmark.y])

            if calibration_started:
                samples_collected += 1
                cv2.putText(image, f"Recopilando muestras: {samples_collected}/{max_calibration_samples}",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Romper el ciclo después de obtener suficientes muestras
                if samples_collected >= max_calibration_samples:
                    break
            else:
                cv2.putText(image, "Presiona 'c' para calibrar", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('Hand Tracking', image)

            key = cv2.waitKey(1)
            if key & 0xFF == ord('c'):
                calibration_started = True

    # Calcular promedio de las muestras de calibración para cada punta de dedo
    calibration_averages = {index: np.mean(samples, axis=0).tolist() for index, samples in calibration_samples.items()}

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

    return calibration_averages



In [2]:
# Ejemplo de uso
calibration_data = calibration()
for index, coordinates in calibration_data.items():
    print(f"Promedio de coordenadas de la punta del dedo {index}: {coordinates}")

Promedio de coordenadas de la punta del dedo 4: [0.5096363925678845, 0.7884968006037136]
Promedio de coordenadas de la punta del dedo 8: [0.6098494558410849, 0.8595920565294072]
Promedio de coordenadas de la punta del dedo 12: [0.65906013620091, 0.8377453617871127]
Promedio de coordenadas de la punta del dedo 16: [0.6819833227019897, 0.7879100786173407]
Promedio de coordenadas de la punta del dedo 20: [0.675327866153921, 0.722103965473685]


In [6]:
print(calibration_data)

{4: [0.41848419411429044, 0.7838132817169715], 8: [0.4920209790098256, 0.8846265562649431], 12: [0.5515185615112042, 0.8841971681035798], 16: [0.5896902248777193, 0.8421829486715382], 20: [0.608495780517315, 0.7651274553660689]}


In [3]:
import mediapipe as mp
import cv2
import numpy as np

limit = 0.0000005
mp_drawing = mp.solutions.drawing_utils
# Modelo de manos
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)


# Instanciamos el modelo de las manos, detection y tracking son los dos modelos que se usan
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
  # Mientas estamos conectados a la webcam
  while cap.isOpened():
    # Leemos cada cuadro, no usamos ret, frame es el cuadro
    ret, frame = cap.read()

    #DETECCION CON MEDIAPIPE
    # Recoloreamos el feed obtenido de cv2 de BGR a RGB para usarlo en mediapipe
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    #Invertimos la imagen de manera horizontal, para detectar de manera correcta cual mano es derecha y cual izquierda
    image = cv2.flip(image,1)
    # Evita que podamos dibujar sobre la imagen
    image.flags.writeable = False
    # Realizamos la deteccion
    results = hands.process(image)
    # Nos permite dibujar sobre la imagen de nuevo
    image.flags.writeable = True

    # Convertimos de RGB a BGR
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    # print(results)

    # Renderizar los resultados si detecta akgi (en la variable results)
    if results.multi_hand_landmarks:
      hand_landmarks = results.multi_hand_landmarks[0]
      for index in [4, 8, 12, 16, 20]:
        landmark = hand_landmarks.landmark[index]
        # Imprimir el estado de tocar el piano basado en la aproximación a los promedios calculados
        if index < len(calibration_data):
            average_x, average_y = calibration_data[index]
            current_x, current_y = landmark.x, landmark.y
            if (average_x * 0.9 <= current_x <= average_x * 1.1) and (average_y * 0.9 <= current_y <= average_y * 1.1):
                print(f'Tocando Piano - Coordenadas (x, y): ({current_x}, {current_y})')
            else:
                print(f'No tocando Piano - Coordenadas (x, y): ({current_x}, {current_y})')

      for num, hand in enumerate(results.multi_hand_landmarks):
        mp_drawing.draw_landmarks(image,hand, mp_hands.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))

    # Mostramos frame
    cv2.imshow('Hand Tracking', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

cap.release()
cv2.destroyAllWindows()


No tocando Piano - Coordenadas (x, y): (0.6401569843292236, 0.6849765181541443)
No tocando Piano - Coordenadas (x, y): (0.6414834856987, 0.6793724894523621)
No tocando Piano - Coordenadas (x, y): (0.6493914723396301, 0.6226738691329956)
No tocando Piano - Coordenadas (x, y): (0.6547067165374756, 0.6105352640151978)
No tocando Piano - Coordenadas (x, y): (0.6597629189491272, 0.6024102568626404)
No tocando Piano - Coordenadas (x, y): (0.658161997795105, 0.5842235684394836)
No tocando Piano - Coordenadas (x, y): (0.6581780910491943, 0.5875244140625)
No tocando Piano - Coordenadas (x, y): (0.6522337198257446, 0.5997945666313171)
No tocando Piano - Coordenadas (x, y): (0.6476057767868042, 0.6120296120643616)
No tocando Piano - Coordenadas (x, y): (0.6479434967041016, 0.6293386816978455)
No tocando Piano - Coordenadas (x, y): (0.6537916660308838, 0.6306484937667847)
No tocando Piano - Coordenadas (x, y): (0.6546273827552795, 0.6297513842582703)
No tocando Piano - Coordenadas (x, y): (0.62230

### Ignorar: 
Basado en Procentaje

In [ ]:
import mediapipe as mp
import cv2

# Parámetros
limit = 0.0000005
threshold_percentage = 10  # Umbral de cambio porcentual para considerar significativo
max_measurements = 10  # Número máximo de mediciones a mantener en la lista

mp_drawing = mp.solutions.drawing_utils
# Modelo de manos
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)

measurements = []  # Lista para almacenar las últimas mediciones en el eje vertical

# Instanciamos el modelo de las manos, detection y tracking son los dos modelos que se usan
with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    # Mientas estamos conectados a la webcam
    while cap.isOpened():
        # Leemos cada cuadro, no usamos ret, frame es el cuadro
        ret, frame = cap.read()

        #DETECCION CON MEDIAPIPE
        # Recoloreamos el feed obtenido de cv2 de BGR a RGB para usarlo en mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #Invertimos la imagen de manera horizontal, para detectar de manera correcta cual mano es derecha y cual izquierda
        image = cv2.flip(image, 1)
        # Evita que podamos dibujar sobre la imagen
        image.flags.writeable = False
        # Realizamos la deteccion
        results = hands.process(image)
        # Nos permite dibujar sobre la imagen de nuevo
        image.flags.writeable = True

        # Convertimos de RGB a BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # print(results)

        # Renderizar los resultados si detecta alguna mano (en la variable results)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                wrist = hand_landmarks.landmark[0]  # Coordenadas de la muñeca (wrist)
                index_finger = hand_landmarks.landmark[8]  # Coordenadas del dedo índice (dato 8)
                
                # Calcular coordenadas relativas al origen (wrist)
                relative_z = index_finger.z - wrist.z
                
                measurements.append(relative_z)  # Agregar la nueva medición a la lista
                
                if len(measurements) >= max_measurements:
                    # Calcular el promedio de las últimas mediciones
                    avg_measurement = sum(measurements[-max_measurements:]) / max_measurements
                    
                    # Calcular la diferencia porcentual entre la última medición y el promedio de las anteriores
                    percentage_change = ((relative_z - avg_measurement) / avg_measurement) * 100
                    
                    if abs(percentage_change) > threshold_percentage:
                        # Si la diferencia porcentual supera el umbral, considerar que se está tocando el piano
                        status = 'Tocando Piano'
                    else:
                        status = ''
                    
                    # Imprimir estado, coordenadas del dedo índice y porcentaje de diferencia en una sola línea
                    print(f'{status} - Coordenadas del dedo índice (x, y, z): ({index_finger.x}, {index_finger.y}, {index_finger.z}) - Porcentaje de diferencia: {percentage_change:.2f}%')

                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                          mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                                          mp_drawing.DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2))

        # Mostramos frame
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# 